In [1]:
from superpixels import export_superpixel_crf_masks_for_dataset
from dataset import Colonoscopy_Dataset
from tools import (
    return_files_in_directory,
    human_sort
    )
from config import *
import os
from config import *
from embeddings import ResnetFeatureExtractor
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision
from tqdm import tqdm
from tools import visualize
from embeddings import get_mean_embeddings
from config import *

In [2]:
# A model wrapper that gets a resnet model and returns the features before the fully connected layer.
resnet = torchvision.models.resnet50(pretrained=True)
resnet.eval()
feature_extract_model = ResnetFeatureExtractor(resnet)
feature_extract_model.to(DEVICE)

ResnetFeatureExtractor(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

In [3]:
DATA_DIR = BASE_DIR + '/data'
image_files = return_files_in_directory(DATA_DIR + "/original", ".tif")
box_files = return_files_in_directory(DATA_DIR + "/boxmasks", ".png")
# Ensure files are in correct order
human_sort(image_files)
human_sort(box_files)
image_files = image_files[:10]
box_files = box_files[:10]
# Now it is important to only take the training dataset so the embeddings from the other sets aren't considered in the 
# mean embedding
# TODO: Check if the random state is persistant across restarts
X_train, X_test, y_train, y_test = train_test_split(image_files, box_files, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11111, random_state=1) # 0.1111 x 0.9 = 0.1

dataset = Colonoscopy_Dataset(X_train, y_train)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

TESTING_DIR = DATA_DIR + "/testing/robust_boxshrink"
EMBEDDING_DIR = DATA_DIR + "/testing/mean_embeddings/"
if not os.path.exists(TESTING_DIR):
    os.makedirs(TESTING_DIR)

if not os.path.exists(EMBEDDING_DIR):
    os.makedirs(EMBEDDING_DIR)

In [4]:
## Get mean embeddings
get_mean_embeddings(data_loader, model=feature_extract_model, embedding_dir=EMBEDDING_DIR)
f = torch.load(EMBEDDING_DIR + "foreground.pt")
b = torch.load(EMBEDDING_DIR + "background.pt")

Epoch 0:  12%|█▎        | 1/8 [00:04<00:32,  4.69s/batch]

In [ ]:
f.shape

tensor([0.1453, 0.1600, 0.4034,  ..., 0.3141, 0.1326, 0.5029])